In [1]:
import numpy as np
from scipy.optimize import linprog
# import scipy.sparse as spr
import itertools
# from scipy.linalg import sqrtm
# from jax.scipy.optimize import minimize
# import jax.numpy as jnp
# import jax



# Bundled choice 

$$ \begin{align*}
    \min_{\lambda, u} & \sum_i u_i  - \sum_{ik} \phi_{i\hat{B}_i,k} \lambda_k \\
    s.t. & \quad u_i \geq  \phi_{iB0} + \sum_k \phi_{iB,k} \lambda_k \quad \text{for all } iB \in I \times 2^Y
\end{align*}$$
Since typically $ \phi_{i\emptyset k}=0 $ for each $k \in [0,K]$ (i.e. the outside option is normilized to zero) we have $u_i \geq 0.$ We also **assume** that $\lambda \geq 0 .$

Since 

$$ \begin{align*}
    \min_{(\lambda, u)\geq 0} & \sum_i u_i  - \sum_k \phi_{i\hat{B}_i,k} \lambda_k \\
    s.t. & \quad u_i - \phi_{iB0} - \sum_k \phi_{iB,k} \lambda_k - s_{iB} = 0  \quad \text{for all } iB \in \mathcal{C} 
\end{align*}$$

In standard form we have the problem
$$ \begin{align*}
    \min_{x \geq 0} & \quad c ^\top x\\
    s.t. & \quad  A x = b 
\end{align*}$$
where
*  $b = \phi^{\mathcal{C}}_{0},$
* $c = ( -\hat{\phi}, \bm{1}_{I}, \bm{0}_{|\mathcal{C}|} ) $ with $ \hat{\phi}_k = \phi_{i\hat{B}_i,k}$ 
* $A = \begin{pmatrix} \Phi^{\mathcal{C}} & \delta^{\mathcal{C}+} & \bm{I}_{|\mathcal{C}|} \end{pmatrix}$ where $\delta^{\mathcal{C}+}_{iB, \ell}  = \bm{1}_{i = \ell}$  is the out-incidence matrix of $\mathcal{C}.$


This problem has $K +I+ |\mathcal{C}| $ variables and $ |\mathcal{C}|$ equality constraints.

### Gross-Substitutes

In [2]:
K = 4
lambda_k = np.array([4,4,4,2])

I = 30
Y = 100
sigma = 1

### generate exogenous data

np.random.seed(1)
# φ_i_y_0 =  np.random.normal(0,1, size = [I,Y]) 
φ_i_y_0 =  np.random.normal(1,1, size = [I])[:,None] * np.random.normal(0,1, size = [Y])[None, :]
φ_i_y_k = np.random.normal(1,1, size=[I,Y,K-1])
# φ_i_y_k = np.ones(I)[:,None,None] * np.random.normal(1,1, size=[Y,K-1])[None,:,:]
# φ_i_y_k[:,:,0] = np.random.normal(1,1, size=[I])[:,None] * np.ones(Y)[None,:]

eps_iy = sigma * np.random.normal(0,1, size = [I,Y])

def φ_k(i,B):
    return np.concatenate( (φ_i_y_k[i,B,:].sum(0), [ - np.sum(B)**(1.5)]))

def Φ_scalar(i,B, lambda_k):
    
    φ_k = np.concatenate( (φ_i_y_k[i,B,:].sum(0), [ - np.sum(B)**(1.5)]))

    return φ_i_y_0[i,B].sum() + np.dot(φ_k, lambda_k)

In [3]:
### greedy
def greedy(i,lambda_k, p_y = np.zeros(Y)):
    B_k =  np.zeros(Y, dtype=bool)
    val = Φ_scalar(i,B_k, lambda_k)
    k = 0 
    while k < Y:
        y_k = None
        max_add_y =  - np.inf
        for y in np.where(1-B_k)[0]:
            val_add_y = Φ_scalar(i,B_k+ np.eye(1,Y,y,dtype=bool)[0] , lambda_k)   - p_y[y] - p_y[B_k].sum() - val
            if val_add_y > max_add_y:
                y_k = y
                max_add_y = val_add_y
            
        if y_k is None or max_add_y < 0:
            break
        val += max_add_y 
        B_k[y_k] = 1
        k += 1
    return B_k , val

In [4]:
B_hat_i = np.zeros([I,Y], dtype= bool)
for i in range(I):
    B_hat_i[i] = greedy(i, lambda_k, - eps_iy[i] )[0]  
print(np.mean(B_hat_i.sum(1)))
print(np.std(B_hat_i.sum(1)))
print(np.max(np.sum(B_hat_i,1)))
print(np.min(np.sum(B_hat_i,1)))

29.2
1.973153144926499
33
25


Estimation

$$ \begin{align*}
    \min_{x \geq 0} & \quad c ^\top x\\
    s.t. & \quad  A x \geq b 
\end{align*}$$
* $c = \begin{pmatrix} -  \phi_{i\hat{B}_i 0}  \\ \bm{1}_I \end{pmatrix}$
* $A = \begin{pmatrix} -\phi & \bm{I}_I \otimes \bm{1}_{2^Y}  \end{pmatrix}$ 
* $ b =  \phi_0$


In [5]:
# ### brute force

# ### Cost

# c = np.ones([K+I])


# phi_hat = 0

# φ_hat_k = np.zeros([I,K])
# for i in range(I):
#     φ_hat_k[i] = φ_k(i,B_hat_i[i])

# phi_hat =  φ_hat_k.sum(0) 

# c[:K] =  - phi_hat

# ### Constraints
# all_bundles = np.array([np.array(seq) for seq in itertools.product([0, 1], repeat= Y)], dtype = bool)

# phi = np.zeros([I,2 ** Y , K])
# for i in range(I):
#     for id , B in enumerate(all_bundles[1:]):
#         phi[i, id , :] = φ_k(i, B)


# A_all = np.block([- phi.reshape( I * (2**Y ), K) ,  np.kron(np.eye(I), np.ones((2**Y ,1)) ) ])


# phi_0 = np.zeros([I,2 ** Y ] )
# for i in range(I):
#     for id, B in enumerate(all_bundles[1:]):
#         phi_0[i, id] = φ_i_y_0[i,B].sum()

# b_all = phi_0.reshape([I * (2 ** Y )])

# print("starting estimation...")

# res = linprog(c, A_ub = - A_all, b_ub =  - b_all, bounds=(None,None), method='highs')
# # print(res)
# print(res.status)
# print(res.x[:K])
# print(res.fun)

### Column generation:

I start with columns $\mathcal{C} = \{i \hat{B}_i\}_i$

In [6]:
φ_hat_k = np.array([φ_k(i,B_hat_i[i]) for i in range(I)])

In [7]:
def minimax_regret(φ_hat_k,B_hat_i, φ_i_y_0, iters):
    c = np.ones([K+I])
    c[:K] =  - φ_hat_k.sum(0)
    b_t = np.array([φ_i_y_0[i,B_hat_i[i]].sum() for i in range(I)])
    A_t =  np.block([- φ_hat_k,np.eye(I)])

    res = linprog(c, A_ub = -A_t, b_ub = -b_t, bounds=(None,None), method='highs')
    # print(res.x)
    iter = 0
    for _ in range(iters):
        B_star_i = []
        val_i = []
        for i in range(I):
            B_star, val = greedy(i,lambda_k= res.x[:K])
            B_star_i.append(B_star)
            val_i.append(val - res.x[  K+ i] ) 
        print(np.max(val_i) )
        if np.max(val_i) < 1e-12:
            print(f"solution found! Iterations: {iter}")
            print(res.fun)
            break
        
        # add to C
        rows_to_add = np.block([ - np.array([φ_k(i,B_star_i[i]) for i in range(I)]),
                                np.eye(I) ])
        A_t = np.block([[A_t],
                        [ rows_to_add] ])
        b_t = np.concatenate((b_t, np.array([φ_i_y_0[i,B_star_i[i]].sum() for i in range(I)])))

        res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')
        # print(res.x[:K] )
        iter += 1
    return res.x[:K]

In [8]:
print(minimax_regret(φ_hat_k,B_hat_i, φ_i_y_0, 30))
print(lambda_k)

80.14927862181396
227.30133972409988
47.467384812326586
80.16534351088887
33.57574769967226
97.5638047820172
75.76837177953854
51.276598008318146
19.735573380187674
25.07966341695763
19.44070820478987
6.097340521208707
12.182426108205163
3.756099555660626
1.9074235352605342
0.8145196434290085
0.5378708452659282
0.024069308112387944
5.684341886080802e-14
solution found! Iterations: 18
358.547312897078
[2.98746049 2.91446381 3.04615907 1.49251838]
[4 4 4 2]


### GMM by simulation

In [9]:
S = 30
# sigma = 1
np.random.seed(987)
eps_is_y = sigma * np.random.normal(0,1, size = [I * S,Y]) 


In [10]:
iters =  50

φ_hat_k = np.array([φ_k(i, B_hat_i[i]) for i in range(I)])
c = np.ones([ K + I * S])/S
c[:K] =  - φ_hat_k.sum(0)

b_t = np.array([φ_i_y_0[i_s // S,B_hat_i[i_s // S]].sum() 
                + eps_is_y[i_s, B_hat_i[i_s // S]].sum()
                for i_s in range(I * S)])
A_t =  np.block([- np.kron(φ_hat_k, np.ones((S, 1))) , np.eye(I * S)])

res = linprog(c, A_ub = -A_t, b_ub = -b_t, bounds=(None,None), method='highs')

iter = 0
for _ in range(iters):
    ### Pricing Problem
    B_star_is = []
    val_is = []
    for i_s in range(I * S):
        B_star, val = greedy(i_s // S , lambda_k = res.x[:K], p_y =  - eps_is_y[i_s])
        B_star_is.append(B_star)
        val_is.append(val - res.x[K + i_s] ) 
    print(np.max(val_is) )
    if np.max(val_is) < 1e-12:
        print(f"solution found! Iterations: {iter}")
        print(res.fun)
        break
    ### Master Problem
    rows_to_add = np.block([ - np.array([φ_k(i_s // S ,B_star_is[i_s]) for i_s in range(I * S)]),
                                np.eye(I * S) ])
    A_t = np.block([[A_t],
                    [rows_to_add] ])
    b_t = np.concatenate((b_t, 
                        np.array([φ_i_y_0[i_s // S , B_star_is[i_s]].sum() 
                                  + eps_is_y[i_s, B_star_is[i_s]].sum() 
                                for i_s in range(I * S)])))

    res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')

    iter += 1

print(res.x[:K])
print(lambda_k)

101.48086959899956
177.97762298013654
59.407877729151934
60.98859701290192
41.576814462548896
19.709989458403328
115.47077729332659
16.317935543460578
7.417036596322134
3.3944149746825474
1.4432814361450141
0.7091017305302785
0.21100899632415349
0.056309175907756526
1.7053025658242404e-13
solution found! Iterations: 14
452.3399653073508
[3.97414917 4.002409   4.04904016 1.99472451]
[4 4 4 2]
